# Bottleneck distance visualisation of PA congressional plan

## Preliminary Steps

Import some packages.

In [4]:
import geopandas as gpd

import random

import matplotlib.pyplot as plt
from functools import partial
import networkx as nx

from gerrychain import MarkovChain
from gerrychain.constraints import (
    Validator,
    single_flip_contiguous,
    within_percent_of_ideal_population,
)
from gerrychain.proposals import propose_random_flip
from gerrychain.accept import always_accept
from gerrychain.updaters import Election, Tally, cut_edges
from gerrychain.partition import Partition
from gerrychain.proposals import recom
from gerrychain.metrics import mean_median, efficiency_gap

import csv
import os
from functools import partial
import json

import matplotlib.pyplot as plt

from gerrychain import (
    Election,
    Graph,
    MarkovChain,
    Partition,
    accept,
    constraints,
    updaters,
)

from gerrychain.tree import recursive_tree_part

Define functions to create district adjacency graphs.

In [5]:
def district_pairs(part):
    pairs = []
    for x, y in part['cut_edges']:
        pairs.append((part.assignment[x],part.assignment[y]))
    return set(pairs)

def adjacency_graph_cut_edges(part):
    edges = district_pairs(part)
    adjacency_graph = nx.Graph()
    adjacency_graph.add_nodes_from(list({x for (x,y) in edges}))
    adjacency_graph.add_edges_from(list(edges))
    return adjacency_graph

Import some data.

In [6]:
pop_col = "TOT_POP"
election_names = [
    "PRES12",
    "PRES16",
    "SENW101216",
]
election_columns = [
    ["PRES12D", "PRES12R"],
    ["T16PRESD", "T16PRESR"],
    ["W101216D", "W101216R"],
]
graph = Graph.from_file("./PA_VTD/PA_VTD.shp")

Define updaters.

In [7]:
updaters1 = {
    "population": updaters.Tally("TOT_POP", alias="population"),
    "cut_edges": cut_edges,
}

elections = [
    Election(
        election_names[i],
        {"Democratic": election_columns[i][0], "Republican": election_columns[i][1]},
    )
    for i in range(len(election_names))
]

election_updaters = {election.name: election for election in elections}

updaters1.update(election_updaters)

## Get some "real" plans

We will look at some 'premade' partitions. In particular, `partition_2011` is the real districting plan from 2011.

In [8]:
#print("Sample node: ", graph.nodes[0])
#fix some strings which should be numbers
for n in graph.nodes():
    graph.nodes[n]['538CPCT__1'] = int(graph.nodes[n]['538CPCT__1'])
    graph.nodes[n]['538DEM_PL'] = int(graph.nodes[n]['538DEM_PL'])
    graph.nodes[n]['538GOP_PL'] = int(graph.nodes[n]['538GOP_PL'])
    graph.nodes[n]['8THGRADE_1'] = int(graph.nodes[n]['8THGRADE_1'])
    

In [9]:
partition_2011 = Partition(graph, "2011_PLA_1", updaters1)
partition_GOV = Partition(graph, "GOV", updaters1)
partition_TS = Partition(graph, "TS", updaters1)
partition_REMEDIAL = Partition(graph, "REMEDIAL_P", updaters1)
partition_CPCT = Partition(graph, "538CPCT__1", updaters1)
partition_DEM = Partition(graph, "538DEM_PL", updaters1)
partition_GOP = Partition(graph, "538GOP_PL", updaters1)
partition_8th = Partition(graph, "8THGRADE_1", updaters1)

In [10]:
ideal_population = sum(partition_2011["population"].values())/len(partition_2011)
print("Ideal popuation: ", ideal_population)

Ideal popuation:  704718.2777777778


## Create Recom Plans

In [11]:
initial_partition = Partition(graph, "GOV", updaters1)

proposal = partial(
    recom, pop_col="TOT_POP", pop_target=ideal_population, epsilon=0.02, node_repeats=2
)

compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]), 2 * len(initial_partition["cut_edges"])
)

recom_steps = 10000

In [12]:
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(initial_partition, 0.02),
        compactness_bound,  # single_flip_contiguous#no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=recom_steps,
)

In [13]:
recom_partitions = []
recom_labels = []
t=0
for part in chain:
    t+=1
    #recom_partitions.append(part)
    rand = random.random()
    if rand < 0.01:
        recom_partitions.append(part)
        recom_labels.append("GOVRecom" + str(t))
    if t % 20 == 0:
        print("Made Recom " + str(t))

Made Recom 20
Made Recom 40
Made Recom 60
Made Recom 80
Made Recom 100
Made Recom 120
Made Recom 140
Made Recom 160
Made Recom 180
Made Recom 200
Made Recom 220
Made Recom 240
Made Recom 260
Made Recom 280
Made Recom 300
Made Recom 320
Made Recom 340
Made Recom 360
Made Recom 380
Made Recom 400
Made Recom 420
Made Recom 440
Made Recom 460
Made Recom 480
Made Recom 500
Made Recom 520
Made Recom 540
Made Recom 560
Made Recom 580
Made Recom 600
Made Recom 620
Made Recom 640
Made Recom 660
Made Recom 680
Made Recom 700
Made Recom 720
Made Recom 740
Made Recom 760
Made Recom 780
Made Recom 800
Made Recom 820
Made Recom 840
Made Recom 860
Made Recom 880
Made Recom 900
Made Recom 920
Made Recom 940
Made Recom 960
Made Recom 980
Made Recom 1000
Made Recom 1020
Made Recom 1040
Made Recom 1060
Made Recom 1080
Made Recom 1100
Made Recom 1120
Made Recom 1140
Made Recom 1160
Made Recom 1180
Made Recom 1200
Made Recom 1220
Made Recom 1240
Made Recom 1260
Made Recom 1280
Made Recom 1300
Made Recom 13

In [14]:
second_partition = Partition(graph, "2011_PLA_1", updaters1)

second_chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(second_partition, 0.02),
        compactness_bound,  # single_flip_contiguous#no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=second_partition,
    total_steps=recom_steps,
)

In [ ]:
second_recom_partitions = []
second_recom_labels = []
t=0
for part in chain:
    t+=1
    #recom_partitions.append(part)
    rand = random.random()
    if rand < 0.01:
        second_recom_partitions.append(part)
        second_recom_labels.append("2011Recom" + str(t))
    if t % 20 == 0:
        print("Made Recom " + str(t))

Made Recom 20
Made Recom 40
Made Recom 60
Made Recom 80
Made Recom 100
Made Recom 120
Made Recom 140
Made Recom 160
Made Recom 180
Made Recom 200
Made Recom 220
Made Recom 240
Made Recom 260
Made Recom 280
Made Recom 300
Made Recom 320
Made Recom 340
Made Recom 360
Made Recom 380
Made Recom 400
Made Recom 420
Made Recom 440
Made Recom 460
Made Recom 480
Made Recom 500
Made Recom 520
Made Recom 540
Made Recom 560
Made Recom 580
Made Recom 600
Made Recom 620
Made Recom 640
Made Recom 660
Made Recom 680
Made Recom 700
Made Recom 720
Made Recom 740
Made Recom 760
Made Recom 780
Made Recom 800
Made Recom 820
Made Recom 840
Made Recom 860
Made Recom 880
Made Recom 900
Made Recom 920
Made Recom 940
Made Recom 960
Made Recom 980
Made Recom 1000
Made Recom 1020
Made Recom 1040
Made Recom 1060
Made Recom 1080
Made Recom 1100
Made Recom 1120
Made Recom 1140
Made Recom 1160
Made Recom 1180
Made Recom 1200
Made Recom 1220
Made Recom 1240
Made Recom 1260
Made Recom 1280
Made Recom 1300
Made Recom 13

In [ ]:
third_partition = Partition(graph, "538DEM_PL", updaters1)

third_chain = MarkovChain(
    proposal=proposal,
    constraints=[
        constraints.within_percent_of_ideal_population(third_partition, 0.02),
        compactness_bound,  # single_flip_contiguous#no_more_discontiguous
    ],
    accept=accept.always_accept,
    initial_state=third_partition,
    total_steps=recom_steps,
)

In [ ]:
third_recom_partitions = []
third_recom_labels = []
t=0
for part in chain:
    t+=1
    #recom_partitions.append(part)
    rand = random.random()
    if rand < 0.01:
        third_recom_partitions.append(part)
        third_recom_labels.append("538DemRecom" + str(t))
    if t % 20 == 0:
        print("Made Recom " + str(t))

## Create tree plans and add them to the list

In [ ]:
tree_partitions = []

tree_plans = 20 #set this higher later...

for i in range(tree_plans):
    print('Finished tree plan', i)
    cddict = recursive_tree_part(graph, range(18), ideal_population, "TOT_POP", .01, 2)
    tree_partitions.append(Partition(graph, cddict, updaters1))

In [ ]:
partition_list = [partition_2011, partition_GOV, partition_TS,
                  partition_REMEDIAL, partition_CPCT, partition_DEM,
                  partition_GOP, partition_8th]

In [ ]:
labels = ['2011', 'GOV', 'TS', 'REMEDIAL', 'CPCT', 'DEM', 'GOP', '8th']+["Tree" + str(k + 1) for k in range(tree_plans)];

In [ ]:
labels += recom_labels
labels += second_recom_labels
labels += third_recom_labels

In [ ]:
#print(labels)
print(len(labels))

In [ ]:
full_partition_list = partition_list + tree_partitions

In [ ]:
full_partition_list += recom_partitions
full_partition_list += second_recom_partitions
full_partition_list += third_recom_partitions
#full_partition_list = [partition_GOV] + recom_partitions
#labels = ['GOV'] + ["Recom" + str(k+1) for k in range(recom_steps)]

In [ ]:
num_partitions = len(full_partition_list)
print(num_partitions)

## Computing Adjacency Graphs and Barcodes

In [ ]:
import gudhi as gd
import numpy as np

In [ ]:
for k in range(num_partitions):
    part = full_partition_list[k]

    adjacency_graph = adjacency_graph_cut_edges(part) #2 districts have edge between them if share boundary

    spCpx = gd.SimplexTree()
    for edge in adjacency_graph.edges:
        spCpx.insert(list(edge))

    Democratic_voter_share = part['PRES12'].percents('Democratic')

    zero_skeleton = spCpx.get_skeleton(0) #all the vertices

    for j in range(len(zero_skeleton)):
        spCpx.assign_filtration(zero_skeleton[j][0], filtration=Democratic_voter_share[j])

    spCpx.make_filtration_non_decreasing() # when have boundary of complex, add it?

    BarCodes = spCpx.persistence()

    fig = plt.figure(figsize = (10,5))

    ax = fig.add_subplot(1,2,1)
    nx.draw_networkx(adjacency_graph, node_color=Democratic_voter_share, cmap=plt.cm.Blues)
    plt.title('Partition '+labels[k])
    ax.axis('off')

    fig.add_subplot(1,2,2)
    gd.plot_persistence_diagram(BarCodes);

## Bottleneck Distances

In [ ]:
partitions_spCpx = []

for k in range(num_partitions):
    part = full_partition_list[k]

    adjacency_graph = adjacency_graph_cut_edges(part)

    spCpx = gd.SimplexTree()
    for edge in adjacency_graph.edges:
        spCpx.insert(list(edge))
    
    Democratic_voter_share = part['PRES12'].percents('Democratic')
    
    zero_skeleton = spCpx.get_skeleton(0)

    for j in range(len(zero_skeleton)):
        spCpx.assign_filtration(zero_skeleton[j][0], filtration=Democratic_voter_share[j])

    spCpx.make_filtration_non_decreasing()

    partitions_spCpx.append(spCpx)

Now we can create a distance matrix containing the bottleneck distance between any pair of barcodes.

In [ ]:
distMat = np.zeros((num_partitions,num_partitions))

for j in range(num_partitions):
    for k in range(num_partitions):
        spCpx0 = partitions_spCpx[j]
        spCpx1 = partitions_spCpx[k]
        
        spCpx0.persistence()
        spCpx1.persistence()

        I0 = spCpx0.persistence_intervals_in_dimension(0)
        I1 = spCpx1.persistence_intervals_in_dimension(0)

        distMat[j,k] = gd.bottleneck_distance(I0,I1)
        
# Symmetrize to remove any numerical errors

In [ ]:
distMat = np.maximum(distMat, distMat.T)

We view the distance matrix as an image.

In [ ]:
plt.imshow(distMat,cmap = 'hot');
plt.axis('off');

To understand the shape of the 'space of districting plans', we can use Multi-Dimensional Scaling. This algorithm looks for the set of points in $\mathbb{R}^2$ (or $\mathbb{R}^3$) whose distance matrix is as close as possible to the distance matrix we just computed. For a more precise description, go here: https://en.wikipedia.org/wiki/Multidimensional_scaling

The result gives us a visualization of how similar the districting plans are.

In [ ]:
colors = len(partition_list)*['r'] + len(tree_partitions)*['b']+len(recom_partitions)*['g'] + len(second_recom_partitions)*['m'] + len(third_recom_partitions)*['c']
#colors = ['r']+len(recom_partitions)*['g']

In [ ]:
# Import a package containing the MDS algorithm and set options for the algorithm
from sklearn import manifold
mds = manifold.MDS(n_components=2, dissimilarity="precomputed")

# Compute MDS and extract the coordinates of the points
results = mds.fit(distMat)
coords = results.embedding_

z = coords[:,0]
y = coords[:,1]

fig, ax = plt.subplots()
ax.scatter(z, y, c = colors)

for i, txt in enumerate(labels):
    ax.annotate(txt, (z[i], y[i]))

In [ ]:
# Import a package containing the MDS algorithm and set options for the algorithm
from sklearn import manifold
mds = manifold.MDS(n_components=2, dissimilarity="precomputed")

# Compute MDS and extract the coordinates of the points
results = mds.fit(distMat)
coords = results.embedding_

z = coords[:,0]
y = coords[:,1]

fig, ax = plt.subplots()
ax.scatter(z, y, c = colors)

We can also use MDS to project the data to 3D. It takes a little more work to make the plot. You can try running this a few times; MDS doesn't have a unique answer and the algorithm involves some randomness, so you will get something different every time you run it.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

mds = manifold.MDS(n_components=3, dissimilarity="precomputed")
results = mds.fit(distMat)
coords = results.embedding_

fig = plt.figure(figsize = (7,7))
ax = Axes3D(fig)

for i, txt in enumerate(labels):
    ax.scatter(coords[i,0],coords[i,1],coords[i,2], c = colors[i]) 
    ax.text(coords[i,0],coords[i,1],coords[i,2],  '%s' % (txt), size=10, zorder=1) 


plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

mds = manifold.MDS(n_components=3, dissimilarity="precomputed")
results = mds.fit(distMat)
coords = results.embedding_

fig = plt.figure(figsize = (7,7))
ax = Axes3D(fig)

for i, txt in enumerate(labels):
    ax.scatter(coords[i,0],coords[i,1],coords[i,2], c = colors[i]) 


plt.show()
plt.savefig('PennsylvaniaRecomExperiment.png')

In [ ]:
from scipy.cluster import hierarchy

linkage = hierarchy.linkage(distMat, 'single')


plt.figure(figsize = (10,10))
hierarchy.dendrogram(linkage, labels = colors);